(C) Joseph Brennan, 2020. All rights reserved.

# Joseph's DIY Covid-19 Dashboard

This dashboard looks at Covid-19 using data from Public Health England.

In [1]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json
from IPython.display import Markdown, display, HTML
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 110

In [3]:
def printmd(string):
    """Prints in markdown format"""
    display(Markdown(string))

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [5]:
with open("my_timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)

In [6]:
def wrangle_data(data):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    global my_timeseriesdf
    my_timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'tests'])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'tests']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(my_timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                my_timeseriesdf.loc[date, column]=value
    my_timeseriesdf.fillna(0.0, inplace=True)
    
    return my_timeseriesdf


df=wrangle_data(data) # df is the dataframe for plotting

In [7]:
filters = [
    'areaType=overview' # note each metric-value pair is inside one string
    ]
structure = {
    "date": "date",
    "tests": "newPillarOneTestsByPublishDate",
    "cases": "newCasesByPublishDate"  
    }

In [8]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
output = wdg.Output() #creates widget output object

@output.capture(clear_output=True,wait=True)

def access_api(button):
    filters = [
    'areaType=overview' # note each metric-value pair is inside one string
    ]
    structure = {
    "date": "date",
    "tests": "newPillarOneTestsByPublishDate",
    "cases": "newCasesByPublishDate"  
    }
    api = Cov19API(filters=filters, structure=structure)
    
    printmd("Downloading latest data...")
    try:
        data = api.get_json()
        printmd("...task completed.")
        now=datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S").split()
        printmd("Data refreshed on " + now[0] + " at " + now[1])
    except:
        printmd("Unable to update data. Please check connection.")
    
    try: 
        with open("my_timeseries.json", "wt") as OUTFILE:
            json.dump(data, OUTFILE)
    except:
        printmd("Unable to update json")
    
    global df1
    df1=wrangle_data(data)
    refresh_graph()

#The Actual Button    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current PHE data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)
output
#Run all cells before clicking

Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

Output()

## Number of New Cases vs Number of New Tests

The Covid Dashboard below displays the number of daily cases against the number of daily tests for the entire UK. New Tests are based on the Government's Pillar One Tests.

Pillar One Tests are defined on the Government website as follows "swab testing in Public Health England (PHE) labs and NHS hospitals for those with a clinical need, and health and care workers".

In [9]:
series=wdg.SelectMultiple(
    options=['cases', 'tests'],
    value=['cases', 'tests'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        my_timeseriesdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
def refresh_graph():
    """ Refresh the graph."""
    current=series.value
    if current==series.options[:]:
        other=[]
        series.value=other
        series.value=current
    else:
        other=series.options[:]
        series.value=other
        
# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

In [10]:
# stack series and scale on top of each other
ctrls=wdg.VBox([series, scale])
# put the graph and the controls side by side
form=wdg.HBox([graph, ctrls])

# Now form contains the entire interface
display(form)

**Author and Copyright Notice** Joseph Brennan: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*